In [2]:
%pip install opencv-python

/bin/bash: /home/kevin.wang/miniconda3/envs/tf-gpu-py310/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 133.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import nrrd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import random

In [ ]:
# test on 18

In [2]:
trainlist = [2, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24,
             25, 26, 27, 29, 31, 32, 
             33, 35, 36, 39, 40, 41, 42, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 
             57, 60, 61, 62, 
             63, 64, 65, 66, 69, 71, 72, 73, 75, 77, 79, 82, 83, 84, 86, 
             87, 88, 90, 91, 92, 93, 94, 95, 96, 97]

In [2]:
# trainlist = [2, 5, 6, 7, 9, 11, 12, 13, 14, 17, 21, 25, 26, 29, 31, 32, 33, 35, 40, 45, 46, 48, 49, 52, 57, 60, 61, 63, 64, 69, 72, 75, 79, 83, 84, 87, 88, 91, 92, 95, 96, 97]

In [3]:
len(trainlist)

69

In [4]:
def random_warp(img, plane):
    (h, w) = img.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    if plane == 'xy':
        theta = random.randint(0, 360)
    else:
        theta = 0
    rotation = cv2.getRotationMatrix2D((cX, cY), theta, 1.0)

    if plane == 'xy': 
        scale_x = np.round(random.uniform(0.75, 1.25), 2)
        scale_y = np.round(random.uniform(0.75, 1.25), 2)
    else:
        scale_x = 1
        scale_y = 1
    scale = np.array([[scale_x, 0, 0],
                    [0, scale_y, 0],
                    [0, 0, 1]])
    scale[0,2] = -((scale[0,0] * w/2) - w/2 )
    scale[1,2] = -((scale[1,1] * h/2) - h/2 )

    
    if plane == 'xy':
        shear_x = np.round(random.uniform(-0.25, 0.25), 2)
        shear_y = np.round(random.uniform(-0.25, 0.25), 2)
    else:
        shear_x = np.round(random.uniform(-0.15, 0.15), 2)
        shear_y = np.round(random.uniform(-0.15, 0.15), 2)

    shear = np.array([[1, shear_x, 0],
                    [shear_y, 1, 0],
                    [0, 0, 1]])
    shear[0,2] = -shear[0,1] * w/2
    shear[1,2] = -shear[1,0] * h/2

    affine = np.matmul(np.matmul(rotation, scale), shear)
    return affine[:2]

In [5]:
for number in trainlist:

    for augmentation in range(0, 3):
        print(number, augmentation)
        ca, _ = nrrd.read("../../CA/"+str(number)+".ca.seg.nrrd")
        dilation_size = 3
        element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2 * dilation_size + 1, 2 * dilation_size + 1), (dilation_size, dilation_size))
        for i in range(0, ca.shape[2]):
            ca[:,:,i] = cv2.dilate(ca[:,:,i], element)
        ca_copy = ca.copy()
        full_ct, _ =  nrrd.read("../../ESUS/"+str(number)+".img.nrrd")
        masked = ca * full_ct
        masked = (masked-np.min(masked)) / (np.max(masked)-np.min(masked))
        gt, _ =  nrrd.read("../../ESUS/"+str(number)+".b.seg.nrrd")
        gt[gt>1] = 1 # make all plaque segments 1 instead of a new number for each one
        
        if augmentation > 0:
        # skip augmentation if theres no plaque in patch
            if np.sum(gt) == 0:
                continue

        if augmentation > 0:
                
            # warp on x y plane
            xy_transform = random_warp(gt[:,:,0], 'xy')
            w, h = gt[:,:,0].shape[1], gt[:,:,0].shape[0]
            
            for i in range(0, gt.shape[2]):
                gt[:,:,i] = cv2.warpAffine(gt[:,:,i], xy_transform, (w, h))
                masked[:,:,i] = cv2.warpAffine(masked[:,:,i], xy_transform, (w, h))
                ca[:,:,i] = cv2.warpAffine(ca[:,:,i], xy_transform, (w, h))
            
            # warp on x z plane
            xz_transform = random_warp(gt[:,0,:], 'xz')
            w, h = gt[:,0,:].shape[1], gt[:,0,:].shape[0]
            
            for i in range(0, gt.shape[1]):
                gt[:,i,:] = cv2.warpAffine(gt[:,i,:], xz_transform, (w, h))
                masked[:,i,:] = cv2.warpAffine(masked[:,i,:], xz_transform, (w, h))
                ca[:,i,:] = cv2.warpAffine(ca[:,i,:], xz_transform, (w, h))
        
    
        current_volume = 1
        for i in range(0, ca.shape[2]):
            if np.sum(ca[:,:,i]) > 0:
                
                contours, _ = cv2.findContours(ca[:,:,i], mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
                for c in contours:
                    try:
                        # compute the center of the contour
                        M = cv2.moments(c)
                        cX = int(M["m10"] / M["m00"])
                        cY = int(M["m01"] / M["m00"])
                        
                        volume = masked[int(cY-32):int(cY+32), int(cX-32):int(cX+32), i:i+64]
                        nrrd.write("./inputs/"+str(number)+"_volume_"+str(current_volume)+"_aug_"+str(augmentation)+".nrrd", volume)
    
                        volume_gt = gt[int(cY-32):int(cY+32), int(cX-32):int(cX+32), i:i+64]
                        nrrd.write("./gt/"+str(number)+"_volume_"+str(current_volume)+"_aug_"+str(augmentation)+".nrrd", volume_gt)
    
                        ca[int(cY-32):int(cY+32), int(cX-32):int(cX+32), i:i+64] = 0
                        current_volume += 1
                    except:
                        pass

2 0
2 1
2 2
5 0
5 1
5 2
6 0
6 1
6 2
7 0
7 1
7 2
9 0
9 1
9 2
10 0
10 1
10 2
11 0
11 1
11 2
12 0
12 1
12 2
13 0
13 1
13 2
14 0
14 1
14 2
15 0
15 1
15 2
16 0
16 1
16 2
17 0
17 1
17 2
21 0
21 1
21 2
22 0
22 1
22 2
23 0
23 1
23 2
24 0
24 1
24 2
25 0
25 1
25 2
26 0
26 1
26 2
27 0
27 1
27 2
29 0
29 1
29 2
31 0
31 1
31 2
32 0
32 1
32 2
33 0
33 1
33 2
35 0
35 1
35 2
36 0
36 1
36 2
39 0
39 1
39 2
40 0
40 1
40 2
41 0
41 1
41 2
42 0
42 1
42 2
45 0
45 1
45 2
46 0
46 1
46 2
47 0
47 1
47 2
48 0
48 1
48 2
49 0
49 1
49 2
50 0
50 1
50 2
51 0
51 1
51 2
52 0
52 1
52 2
53 0
53 1
53 2
55 0
55 1
55 2
57 0
57 1
57 2
60 0
60 1
60 2
61 0
61 1
61 2
62 0
62 1
62 2
63 0
63 1
63 2
64 0
64 1
64 2
65 0
65 1
65 2
66 0
66 1
66 2
69 0
69 1
69 2
71 0
71 1
71 2
72 0
72 1
72 2
73 0
73 1
73 2
75 0
75 1
75 2
77 0
77 1
77 2
79 0
79 1
79 2
82 0
82 1
82 2
83 0
83 1
83 2
84 0
84 1
84 2
86 0
86 1
86 2
87 0
87 1
87 2
88 0
88 1
88 2
90 0
90 1
90 2
91 0
91 1
91 2
92 0
92 1
92 2
93 0
93 1
93 2
94 0
94 1
94 2
95 0
95 1
95 2
96 0
96 1


In [6]:
trainlist = os.listdir("./gt")

In [7]:
trainlist

['21_volume_10_aug_0.nrrd',
 '47_volume_6_aug_1.nrrd',
 '27_volume_90_aug_1.nrrd',
 '23_volume_17_aug_2.nrrd',
 '97_volume_8_aug_2.nrrd',
 '23_volume_99_aug_0.nrrd',
 '12_volume_10_aug_0.nrrd',
 '23_volume_54_aug_2.nrrd',
 '73_volume_25_aug_0.nrrd',
 '77_volume_28_aug_1.nrrd',
 '27_volume_51_aug_1.nrrd',
 '73_volume_26_aug_1.nrrd',
 '50_volume_30_aug_1.nrrd',
 '71_volume_23_aug_2.nrrd',
 '62_volume_20_aug_1.nrrd',
 '66_volume_12_aug_1.nrrd',
 '9_volume_6_aug_2.nrrd',
 '90_volume_59_aug_2.nrrd',
 '65_volume_10_aug_0.nrrd',
 '21_volume_8_aug_0.nrrd',
 '42_volume_8_aug_1.nrrd',
 '42_volume_1_aug_0.nrrd',
 '22_volume_5_aug_0.nrrd',
 '36_volume_5_aug_1.nrrd',
 '23_volume_71_aug_2.nrrd',
 '71_volume_33_aug_1.nrrd',
 '75_volume_1_aug_1.nrrd',
 '90_volume_13_aug_1.nrrd',
 '90_volume_34_aug_2.nrrd',
 '73_volume_130_aug_0.nrrd',
 '36_volume_37_aug_2.nrrd',
 '10_volume_7_aug_0.nrrd',
 '60_volume_5_aug_2.nrrd',
 '15_volume_2_aug_0.nrrd',
 '16_volume_12_aug_2.nrrd',
 '90_volume_10_aug_2.nrrd',
 '62